# Notebook: Merge/Filter Annotations

## Packages

In [20]:
import pandas as pd
import ast
import os

## Parameters

In [21]:
PATH_ANNOTATIONS = "annotations/"

## Code

### 1. Check if every review got annotated

In [22]:
import os
import pandas as pd

annotator_counts = {}

for filename in os.listdir(PATH_ANNOTATIONS):
    if filename.endswith('.csv'):
        file_path = os.path.join(PATH_ANNOTATIONS, filename)
        df = pd.read_csv(file_path)
        annotator_column = df['annotator']
        
        for annotator in annotator_column:
            if annotator in annotator_counts:
                annotator_counts[annotator] += 1
            else:
                annotator_counts[annotator] = 1

for annotator, count in annotator_counts.items():
    print(f'{annotator}: {count} times')

nils-constantin.hellwig@stud.uni-regensburg.de: 18 times


### 2. Merge Annotations

Annotations are merged if:
* a: same aspect-phrase annotated
* b: same implicit aspect annotated

In [34]:
def convert_label_string_to_dict(label_str):
    try:
        return ast.literal_eval(label_str)
    except (SyntaxError, ValueError):
        return {}
    
df1 = pd.read_csv(PATH_ANNOTATIONS + "annotation_1.csv")
df2 = pd.read_csv(PATH_ANNOTATIONS + "annotation_2.csv")
df1['label'] = df1['label'].apply(convert_label_string_to_dict)
df2['label'] = df2['label'].apply(convert_label_string_to_dict)

In [39]:
df1

,agreement,annotation_id,annotator,created_at,id,label,label-no-phrase,lead_time,restaurant_id,review_id,text,updated_at
0,100.0,21817284,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:25:30.147961Z,68133812,"[{'end': 15, 'text': 'Wartezeit', 'start': 6, ...",NaN,167.779,1870837,870392222,"Lange Wartezeit auf einen Tisch, das Restauran...",2023-09-07T08:25:30.147981Z
1,100.0,21817293,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:25:55.379940Z,68133813,"[{'end': 10, 'text': 'Steaks', 'start': 4, 'la...",NaN,32.799,1308296,887012563,Die Steaks waren sehr gut auch die Kartoffeln ...,2023-09-07T08:26:38.238132Z
2,100.0,21817344,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:29:24.013168Z,68133814,"[{'end': 7, 'text': 'Fritten', 'start': 0, 'la...",NaN,157.676,10685715,849351269,Fritten waren gut und wir haben uns eine große...,2023-09-07T08:29:24.013193Z
3,100.0,21817406,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:33:11.592063Z,68133815,"[{'end': 25, 'text': 'Toiletten', 'start': 16,...","[{""end"":101,""text"":""aber sauber u"",""start"":88,...",226.666,11895711,881137672,Leider sind die Toiletten eine Etage höher und...,2023-09-07T08:33:11.592093Z
4,100.0,21817410,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:33:18.043902Z,68133816,"[{'end': 12, 'text': 'Essen', 'start': 7, 'lab...",NaN,5.667,5967363,871925315,Tolles Essen!,2023-09-07T08:33:18.043922Z
5,100.0,21817476,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:36:13.822842Z,68133817,{},NaN,174.900,10130718,882916333,Die Hälfte habe ich übrig gelassen!,2023-09-07T08:36:13.822862Z
6,100.0,21817508,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:37:10.825548Z,68133818,"[{'end': 21, 'text': 'Pizza', 'start': 16, 'la...",NaN,56.172,801344,885335081,Wir hatten eine Pizza bestellt die vom Belag o...,2023-09-07T08:37:10.825570Z
7,100.0,21817530,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:37:58.146691Z,68133819,"[{'end': 9, 'text': 'Tisch', 'start': 4, 'labe...",NaN,46.477,6949511,875208633,"Der Tisch, der uns dann, wiederum mit einem Fi...",2023-09-07T08:37:58.146712Z
8,100.0,21817933,nils-constantin.hellwig@stud.uni-regensburg.de,2023-09-07T08:46:18.399200Z,68133820,"[{'end': 30, 'text': 'Preis-Leistungs-Verhältn...",NaN,499.446,6914832,861065137,Das Preis-Leistungs-Verhältnis stimmt nicht me...,2023-09-07T08:46:18.399222Z


duplikate entfernen -> evtl kommt das mal vor

In [42]:
matching_rows = pd.DataFrame([])

for idx in range(len(df1)):
    equal_explicit_annotation = []
    for item1 in df1['label'].iloc[idx]:
        for item2 in df2['label'].iloc[idx]:
            if item1 == item2:
                equal_explicit_annotation.append(item1)
    matching_rows = pd.concat([matching_rows, pd.DataFrame({
        'label_explicit': equal_explicit_annotation,
        'id': df1['id'].iloc[idx],
        'restaurant_id': df1['restaurant_id'].iloc[idx],
        'review_id': df1['review_id'].iloc[idx],
        'text': df1['text'].iloc[idx],
    })], ignore_index=True)

matching_rows

,label_explicit,id,restaurant_id,review_id,text
0,"{'end': 15, 'text': 'Wartezeit', 'start': 6, '...",68133812,1870837,870392222,"Lange Wartezeit auf einen Tisch, das Restauran..."
1,"{'end': 47, 'text': 'Restaurant', 'start': 37,...",68133812,1870837,870392222,"Lange Wartezeit auf einen Tisch, das Restauran..."
2,"{'end': 45, 'text': 'Kartoffeln', 'start': 35,...",68133813,1308296,887012563,Die Steaks waren sehr gut auch die Kartoffeln ...
3,"{'end': 59, 'text': 'Salat', 'start': 54, 'lab...",68133813,1308296,887012563,Die Steaks waren sehr gut auch die Kartoffeln ...
4,"{'end': 7, 'text': 'Fritten', 'start': 0, 'lab...",68133814,10685715,849351269,Fritten waren gut und wir haben uns eine große...
5,"{'end': 54, 'text': 'Portion', 'start': 47, 'l...",68133814,10685715,849351269,Fritten waren gut und wir haben uns eine große...
6,"{'end': 25, 'text': 'Toiletten', 'start': 16, ...",68133815,11895711,881137672,Leider sind die Toiletten eine Etage höher und...
7,"{'end': 12, 'text': 'Essen', 'start': 7, 'labe...",68133816,5967363,871925315,Tolles Essen!
8,"{'end': 21, 'text': 'Pizza', 'start': 16, 'lab...",68133818,801344,885335081,Wir hatten eine Pizza bestellt die vom Belag o...
9,"{'end': 76, 'text': 'Gewürze', 'start': 69, 'l...",68133818,801344,885335081,Wir hatten eine Pizza bestellt die vom Belag o...


### To-do: Filter annotations with mixed label 

### To-do: Filter annotations with no label